In [12]:
include("../src/geosteering.jl");
include("../src/mdp.jl");
include("../src/utils.jl");

#set seed for reproducibility
rng = MersenneTwister(1) 

## Case with tiny drifting probability

In [ ]:
#initialize the MDP model
gs = initialize_mdp(
    rng=rng, size=(5, 5), 
    base_amplitude=1.0, 
    base_frequency=1.0,
    target_thickness=2.0, 
    vertical_shift=3.0,
    drift_prob=0.001) #no drift_prob

In [ ]:
# one simulation step example
p0 = initialstate(gs) # initial state distribution
s = rand(rng, p0) # initial state s ~ p0

#plot the initial state
render(gs, (s=s,))    

In this 5x5 grid problem, white cells mean target zone, light gray cells mean shale zone, and darker gray cells mean harder rock. The goal is to determine the sequence of drill directions (forward, top, or bottom) from orange circle location to other end of the target zone to maximize surface coverage of the drill path.

## MDP Model Components

### State Space

The state $s$ is simply the location (x, y) and binaries whether the surface area has been drilled or not. 

In [ ]:
𝒮 = states(gs)

#example of states
[println(𝒮[i]) for i in 1:3] 

println("The size of the state space is: ", length(𝒮))

### Action space

The action space is relatively small, with only 4 possible actions: move forward (`RIGHT`), `UP` or `DOWN`.

In [ ]:
𝒜 = actions(gs)

### Transition function

The transition function in this problem is simple: whenever we take a drill action, we move toward the direction of that action with probability $1-p_\text{drift}$, and got drifted to random location with probability $p_\text{drift}$. However, we aren't able to go out of bounds of the system.

Also, the first time we visit a new surface area (new x-coordinate), we set the Boolean visited status of that surface area to be `true`, otherwise it stays at default value `false`

We can simulate a state transition as follows.

In [ ]:
# we can take random action from the action space

a = rand(rng, 𝒜) # random action
p = transition(gs, s, a) # get next state distribution p(s' | s, a)
sp = rand(rng, p) # the next state is sampled from this s ~ p

println("From state s with location ", s.cell, " and surrounding indicators (of being within formation/not):")
[println(key, ": ", value) for (key, value) in s.is_surrounding_target];
println("to state sp with location ", sp.cell, " and surroundings:")
[println(key, ": ", value) for (key, value) in sp.is_surrounding_target];

Had we taken an action RIGHT, we will have the following:

In [ ]:
a_alt = DOWN
p_alt = transition(gs, s, a_alt) # get next state distribution p(s' | s, a)
sp_alt = rand(rng, p_alt) # the next state is sampled from this s ~ p

println("The new state sp with location ", sp.cell, " and surroundings:")
[println(key, ": ", value) for (key, value) in sp.is_surrounding_target];

### Reward Function

The reward function `reward(s,a)` or `reward(s,a,sp)` tells the reward we get if from state `s` we take an action `a` and end up at next state `sp`. 

In [ ]:
r = reward(gs, s, a, sp)
println("The reward for taking action ", a, " from state ", s.cell, " and ending up in state ", sp.cell, " is ", r)

r_alt = reward(gs, s, a_alt, sp_alt)
println("The reward for taking action ", a_alt, " from state ", s.cell, " and ending up in state ", sp_alt.cell, " is ", r_alt)

### Solving the model

With this model, we can try a bunch of approaches to come up with a policy. For MDP, a policy is simply a mapping from state to action. You can think of it as a look up table that prescribes the optimal action to take at every state that maximize our objective.

For this problem, we can formulate the objective as maximizing long-term cumulative reward of staying in the target zone:

\begin{align}
\text{maximize}~ \sum_{t=1}^T r_t \gamma^{t-1},
\end{align}

where $r_t$ is the reward at time $t$ and $\gamma \in [0, 1]$ is the discount factor that reflects how much we weigh more immediate rewards vs. future ones.

We can compare two types of solvers to obtain such an optimized policy: offline solver and online solver.

In [ ]:
#offline planner example
using DiscreteValueIteration

solver = ValueIterationSolver(max_iterations=100);
@time policy = solve(solver, gs);

In [ ]:
# simulate the policy
hr = HistoryRecorder(max_steps=30, rng=rng)
hist = simulate(hr, gs, policy)

# render and save the plots as png images and gif animation
plot_sim_steps = render(gs, hist);
num_steps = length(plot_sim_steps)
[savefig(plot_sim_steps[i], gs.size, "../figs/SimRollout$i.png") for i in 1:num_steps];
create_gif_from_images(dir="../figs/", gif_name="MDPPolicy1_NoDrifting.gif", fps=2, num_steps=num_steps) 

In [ ]:
#online planner example
using MCTS

hr = HistoryRecorder(max_steps=30, rng=rng)
solver = MCTSSolver(n_iterations=1000, depth=30, rng=rng)
@time mcts_policy = solve(solver, gs);

#simulate and render the policy
@time hist = simulate(hr, gs, mcts_policy)

# render and save the plots as png images and gif animation
plot_sim_steps = render(gs, hist);
num_steps = length(plot_sim_steps)
[savefig(plot_sim_steps[i], gs.size, "../figs/SimRollout$i.png") for i in 1:num_steps];
create_gif_from_images(dir="../figs/", gif_name="MDPPolicy2_NoDrifting.gif", fps=2, num_steps=num_steps) 

In this example, we see that the offline solver (`Discrete Value Iteration`) runs much longer compared to the online solver (`Monte Carlo Tree Search or MCTS`), but with better policy (higher objective).  In general, offline solvers oftentimes solve for a more optimal policy (with higher objective), but runs much longer compared to the online solvers (but often subotimal or has lower objective). This is the tradeoff that we have to make in practice, especially as we deal with larger state and action spaces.

## Case with Drifting Probability = 0.25

gs = initialize_mdp(
    rng=rng, size=(5, 5), 
    base_amplitude=1.0, 
    base_frequency=1.0,
    target_thickness=2.0, 
    vertical_shift=3.0,
    drift_prob=0.25) #some drifting prob

In [ ]:
## offline solver with drifting
solver = ValueIterationSolver(max_iterations=100);
@time policy = solve(solver, gs);

#simulate 
hist = simulate(hr, gs, policy)

# render 
plot_sim_steps = render(gs, hist);
num_steps = length(plot_sim_steps)
[savefig(plot_sim_steps[i], gs.size, "../figs/SimRollout$i.png") for i in 1:num_steps];
create_gif_from_images(dir="../figs/", gif_name="MDPPolicy1_WithDrifting.gif", fps=2, num_steps=num_steps) 

In [ ]:
## online solver with drifting
solver = MCTSSolver(n_iterations=1000, depth=30, rng=rng)
@time mcts_policy = solve(solver, gs);

#simulate and render the policy
@time hist = simulate(hr, gs, mcts_policy)

# render 
plot_sim_steps = render(gs, hist);
num_steps = length(plot_sim_steps)
[savefig(plot_sim_steps[i], gs.size, "../figs/SimRollout$i.png") for i in 1:num_steps];
create_gif_from_images(dir="../figs/", gif_name="MDPPolicy2_WithDrifting.gif", fps=2, num_steps=num_steps) 